Agents are connected but have their own states. Only the final responses are appended to global state

<img src="../../resources/images/multi_agent_supervisor.png" alt="Multi Agent Supervisor" height="300" width="300">

Agents are independent LangChain agents. This means they have their own individual prompt, LLM, and tools.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
from src.langchain.lg_workflow import *
from src.langchain.agents.framenet_agent import *
config = {"configurable" : {"thread_id" : 1}}

In [2]:
fnr = framenet_tool.invoke({'instruction':'pick up the cup'})
fnr

INSIDE FRAMENET TOOL


'{\n  "framenet": "picking_up",\n  "frame": "Getting",\n  "lexical-unit": "pick up.v",\n  "core": {\n    "agent": "robot",\n    "theme_patient": "cup",\n    "instrument": "robot gripper",\n    "source": "",\n    "goal": "",\n    "result": "robot has the cup"\n  },\n  "peripheral": {\n    "location": "kitchen workspace",\n    "manner": "gently",\n    "direction": "upward",\n    "time": "during cleanup phase",\n    "quantity": "one item",\n    "portion": ""\n  }\n}'

In [3]:
# Example: Complex Query Using Multiple Agents
input_question = "what is 2 plus 2"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True,
    config=config,
):
    print(s)
    print("----")


Next Worker: mather
((), {'supervisor': None})
----
INSIDE ADD TOOL
(('mather:451715db-5fa9-0259-8b92-e3776d81c411',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:4b', 'created_at': '2025-05-14T13:37:12.709533427Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11444325153, 'load_duration': 10142083, 'prompt_eval_count': 227, 'prompt_eval_duration': 2376090133, 'eval_count': 140, 'eval_duration': 9049944467, 'model_name': 'qwen3:4b'}, id='run--7b927f38-5079-42f9-a3ff-57e9d90f48f3-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': '4f182ee9-a4bd-49d4-a779-a184189a58e0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 227, 'output_tokens': 140, 'total_tokens': 367})]}})
----
(('mather:451715db-5fa9-0259-8b92-e3776d81c411',), {'tools': {'messages': [ToolMessage(content='4.0', name='add', id='35693c29-3ec9-4b4c-8641-aa20412d547f', tool_call_id='4f182ee9-a4bd-49d4-a779-a184189a58e0')]}})
----
(('mather:4

In [3]:
graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='what is 2 plus 2', additional_kwargs={}, response_metadata={}, id='dc8c3d7a-8957-4a0d-a4f1-2ac8357f33c7'), HumanMessage(content='<think>\nOkay, the user asked "what is 2 plus 2". I need to use the add function since that\'s for adding numbers. The parameters are a and b, both numbers. So I\'ll call add with a=2 and b=2. The response from the tool was 4.0, which makes sense. I should present that as the answer clearly.\n</think>\n\nThe result of 2 plus 2 is 4.0.', additional_kwargs={}, response_metadata={}, name='mather', id='74bc0f36-7701-45f2-85c6-e535f3c894b5')]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f030a4e-32de-6ce1-8003-df93608a74e6'}}, metadata={'source': 'loop', 'writes': {'supervisor': None}, 'step': 3, 'parents': {}, 'thread_id': 1}, created_at='2025-05-14T09:21:59.269232+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': 

In [2]:
# Example: Complex Query Using Multiple Agents
input_question = "framenet representation of the action pick up the blue mug"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True,
    config=config,
):
    print(s)
    print("----")


Next Worker: framenet
((), {'supervisor': None})
----
INSIDE FRAMENET TOOL(('framenet:4bf7d798-48d0-cac5-fd0d-3ec8751b5eab',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:4b', 'created_at': '2025-05-14T13:49:27.3412604Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3217029803, 'load_duration': 8288274, 'prompt_eval_count': 188, 'prompt_eval_duration': 1199045476, 'eval_count': 32, 'eval_duration': 2004830480, 'model_name': 'qwen3:4b'}, id='run--3d05272f-6302-493d-8209-494b0d084677-0', tool_calls=[{'name': 'framenet_tool', 'args': {'__arg1': 'pick up the blue mug'}, 'id': 'c9893131-9396-4a10-a5f5-79d987c1c7fa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 188, 'output_tokens': 32, 'total_tokens': 220})]}})
----

(('framenet:4bf7d798-48d0-cac5-fd0d-3ec8751b5eab',), {'tools': {'messages': [ToolMessage(content='"{\\n  \\"framenet\\": \\"picking_up\\",\\n  \\"frame\\": \\"Getting\\",\\n  \\"lexical-unit\\": \\"p

In [3]:
graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='framenet representation of the action pick up the blue mug', additional_kwargs={}, response_metadata={}, id='0799e26c-ec5e-4192-a572-da7977e29718'), HumanMessage(content='<think>\n\n</think>\n\nThe frame net representation of the action "pick up the blue mug" is as follows:\n\n```\n{\n  "framenet": "picking_up",\n  "frame": "Getting",\n  "lexical-unit": "pick up.v",\n  "core": {\n    "agent": "robot",\n    "theme_patient": "blue mug",\n    "instrument": "robot gripper",\n    "source": "",\n    "goal": "",\n    "result": "robot has the blue mug"\n  },\n  "peripheral": {\n    "location": "kitchen workspace",\n    "manner": "gently",\n    "direction": "upward",\n    "time": "during cleanup phase",\n    "quantity": "one item",\n    "portion": ""\n  }\n}\n```', additional_kwargs={}, response_metadata={}, name='framenet', id='27431628-c099-4b72-8ef7-bc18553b76f5')]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': 

In [2]:
graph.invoke({'messages': [HumanMessage(content='framenet representation of the action pick up the cup from the fridge')]}, config=config)['messages'][0].content

Next Worker: framenet
INSIDE FRAMENET TOOL
Next Worker: FINISH


'framenet representation of the action pick up the cup from the fridge'

In [3]:
graph.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='framenet representation of the action pick up the cup from the fridge', additional_kwargs={}, response_metadata={}, id='f483963d-5427-4948-a7e7-58fb2ff3e523'), HumanMessage(content='<think>\nOkay, the user asked for the FrameNet representation of "pick up the cup from the fridge." I called the framenet_tool with that phrase. The response came back with a structured JSON. Let me parse this.\n\nThe main action is "picking_up" under the "Getting" frame. The lexical unit is "pick up.v." The core elements have the agent as "robot," theme_patient as "cup," instrument as "robot gripper," source as "fridge." The result is the robot having the cup. Peripheral details include location, manner, direction, time, etc.\n\nI need to present this in a clear way. Maybe list the main components first, then the peripheral. Make sure to highlight the key elements like agent, theme, instrument, source. Also mention the result. The peripheral info add

In [4]:
# graph.get_state(config=config).values['messages'][-1].content
framenet_answers

['{\n  "framenet": "picking_up",\n  "frame": "Getting",\n  "lexical-unit": "pick up.v",\n  "core": {\n    "agent": "robot",\n    "theme_patient": "cup",\n    "instrument": "robot gripper",\n    "source": "fridge",\n    "goal": "",\n    "result": "robot has the cup"\n  },\n  "peripheral": {\n    "location": "kitchen workspace",\n    "manner": "gently",\n    "direction": "upward",\n    "time": "during cleanup phase",\n    "quantity": "one item",\n    "portion": ""\n  }\n}']

In [13]:
graph.invoke({'messages': [HumanMessage(content='framenet representation of the action pick up the mug from the fridge')]})['messages'][1].content

Next Worker: framenet
INSIDE FRAMENET TOOL
Next Worker: FINISH


'<think>\nOkay, the user asked for the FrameNet representation of "pick up the mug from the fridge." I called the framenet_tool with that instruction. The response came back with the frame "Getting" and the lexical unit "pick up.v". The core elements have agent as robot, theme_patient as mug, instrument as robot gripper, source as fridge, and the result is the robot having the mug. Peripheral details include location, manner, direction, etc.\n\nNow, I need to present this information as the final answer. The user probably wants a clear, structured output without extra explanations. Let me format the response using the FrameNet elements provided. Make sure to include all the core and peripheral elements as listed. Use the exact terms from the tool\'s response. Avoid any markdown and just present the data in a readable way. Check if all the keys and values are correctly transcribed. Alright, that should do it.\n</think>\n\n{\n  "framenet": "picking_up",\n  "frame": "Getting",\n  "lexical

In [3]:
framenet_agent.invoke({'messages': [HumanMessage(content='pick up the mug from the sink')]})

ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'checkpoint_ns', 'checkpoint_id']

In [ ]:
from IPython.display import display, Image
# display(Image(math_agent.get_graph().draw_mermaid_png()))
# display(Image(websearch_agent.get_graph().draw_mermaid_png()))
# display(Image(framenet_agent.get_graph().draw_mermaid_png()))
# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Example: Complex Query Using Multiple Agents
input_question = "what is 2 times 2"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True
):
    print(s)
    print("----")


In [ ]:
# Example: Complex Query Using Multiple Agents
input_question = "what is the value of 2 + 10"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True
):
    print(s)
    print("----")


In [ ]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is apple company founder"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")


In [1]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is the ceo of google and what is the framenet representation of apple"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")


In [2]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is the ceo of google, what is the framenet representation of apple and how much is 2 times 10"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")
